In [21]:
import cobra
from cobra.io import load_json_model
from cobra import Model, Reaction, Metabolite

import pandas as pd
import numpy as np

import escher
from escher import Builder

# 1. Glucose

### Predict fluxes
Synthetic medium from https://www.sciencedirect.com/science/article/pii/S0014579307007235
<br>"minimal synthetic medium containing 48 mM Na2HPO4, 22 mM KH2PO4, 9 mM NaCl, 19 mM NH4Cl, 2 mM MgSO4, 0.1 mM CaCl2, 0.1 g/l of thiamine and 2 g/l of glucose"

In [22]:
# Load the model
model = cobra.io.load_json_model('../data/models/CP022686.1_cur_4.7.json')

In [23]:
wt_solution = model.optimize()
growth = wt_solution.fluxes["BIOMASS_EcN_iHM1533_core_59p80M"]
acetate_production = wt_solution.fluxes["EX_ac_e"]

In [48]:
# Load the model
EcN_model = cobra.io.load_json_model('../data/models/CP022686.1_cur_4.7.json')

# Define the biolog media
medium = EcN_model.medium
medium['EX_glc__D_e'] = 15
    
medium['EX_nh4_e'] = 19 # 19 mM NH4Cl
medium['EX_pi_e'] = 60 # 48mM Na2HPO4 + 22 mM KH2PO4  
medium['EX_so4_e'] = 2 # 2 mM MgSO4
    
medium['EX_mg2_e'] = 2 # 2 mM MgSO4
medium['EX_ca2_e'] = 0.1 # 0.1 mM CaCl2SS
    
medium['EX_na1_e'] = 105 # 48 mM Na2HPO4 + 9 mM NaCl
medium['EX_k_e'] = 22 # 22 mM KH2PO4
medium['EX_cl_e'] = 28 # 9 mM NaCl + 19 mM NH4Cl
    
medium['EX_thm_e'] = 0.38 # 0.1 g/L > 0.1/265.355 x 1000 = 0.376854 mM

# trace elements
value = 0.01
medium['EX_fe3_e'] = value
medium['EX_mn2_e'] = value
medium['EX_fe2_e'] = value
medium['EX_zn2_e'] = value
medium['EX_ni2_e'] = value
medium['EX_cu2_e'] = value
medium['EX_cobalt2_e'] = value
medium['EX_sel_e'] = value
medium['EX_mobd_e'] = value
medium['EX_tungs_e'] = value
medium['EX_slnt_e'] = value
medium['EX_salchsx_e'] = value

EcN_model.medium = medium

In [8]:
# Run parsimonius FBA
sol_glc = cobra.flux_analysis.pfba(EcN_model)

# Export to dataframe
sol_glc_df = sol_glc.to_frame()

### Import flux data

In [25]:
# Import the flux file and set index
flux_df = pd.read_excel('../tables/13C_fluxes.xlsx', usecols=(0,1,2,3))
flux_df = flux_df.dropna()
flux_df.set_index('ID', inplace=True)
flux_df.head()

,Reaction,glc_flux_mean,glc_flux_sd
ID,,,
AKGDH,AKG → Suc + CO2,0.523333,0.008505
FBA,FBP ↔ GAP + GAP,0.715000,0.025632
TPI,FBP ↔ GAP + GAP,0.715000,0.025632
CS,OAA + AcCoA → Cit,0.592333,0.008505
EDA,6PG → GAP + Pyr,0.166667,0.037166


In [50]:
# Merge dataframes
glc_flux_df = pd.merge(flux_df, sol_glc_df, left_index = True, right_index=True)
glc_flux_df['fluxes_corrected'] = glc_flux_df['fluxes']/15 # normalise for 15 glucose
glc_flux_df.drop(columns=['fluxes', 'reduced_costs'], inplace=True) # Do not remove when normalising the 13C data
glc_flux_df

# Alternatively compare the flux with the corrected 13C data
# glc_flux_df['glc_flux_mean_corrected'] = glc_flux_df['glc_flux_mean']*15 # normalise for 15 glucose
# glc_flux_df['glc_flux_sd_corrected'] = glc_flux_df['glc_flux_sd']*15 # normalise for 15 glucose
# glc_flux_df

# Add the subsystem for each reaction and sort
for rxn_id in glc_flux_df.index.values:
    glc_flux_df.loc[rxn_id, 'Subsystem'] = EcN_model.reactions.get_by_id(rxn_id).subsystem
glc_flux_df.sort_values('Subsystem')

,Reaction,glc_flux_mean,glc_flux_sd,fluxes_corrected,Subsystem
DMALRED,Mal ↔ OAA,0.457667,0.01365,0.087308,
ME2,Mal → Pyr + CO2,0.066000,0.008185,0.000000,Anaplerotic Reactions
ME1,Mal → Pyr + CO2,0.066000,0.008185,0.000000,Anaplerotic Reactions
PPC,PEP + CO2 ↔ OAA,0.260000,0.01253,0.225688,Anaplerotic Reactions
AKGDH,AKG → Suc + CO2,0.523333,0.008505,0.007831,Citric Acid Cycle
ICDHyr,Cit → AKG + CO2,0.592333,0.008505,0.089804,Citric Acid Cycle
MDH,Mal ↔ OAA,0.457667,0.01365,0.000000,Citric Acid Cycle
CS,OAA + AcCoA → Cit,0.592333,0.008505,0.089804,Citric Acid Cycle
FUM,Suc → Mal,0.523333,0.007572,0.087308,Citric Acid Cycle
ENO,PGA ↔ PEP,1.511000,0.025632,1.532352,Glycolysis/Gluconeogenesis


- Instead of MDH all flow goes through DMALRED (which includes the formation of FADH2 instead of NADH).
- FBA/TPI shows the flow towards g3p_c/GAP

In [51]:
glc_flux_df['glx_flux_x_15'] = glc_flux_df['glc_flux_mean']*15 # normalise for 15 glucose
glc_flux_df['glx_sd_x_15'] = glc_flux_df['glc_flux_sd']*15 # normalise for 15 glucose
glc_flux_df.sort_values('Subsystem')

,Reaction,glc_flux_mean,glc_flux_sd,fluxes_corrected,Subsystem,glx_flux_x_15,glx_sd_x_15
DMALRED,Mal ↔ OAA,0.457667,0.01365,0.087308,,6.865,0.204756
ME2,Mal → Pyr + CO2,0.066000,0.008185,0.000000,Anaplerotic Reactions,0.990,0.12278
ME1,Mal → Pyr + CO2,0.066000,0.008185,0.000000,Anaplerotic Reactions,0.990,0.12278
PPC,PEP + CO2 ↔ OAA,0.260000,0.01253,0.225688,Anaplerotic Reactions,3.900,0.187949
AKGDH,AKG → Suc + CO2,0.523333,0.008505,0.007831,Citric Acid Cycle,7.850,0.127574
ICDHyr,Cit → AKG + CO2,0.592333,0.008505,0.089804,Citric Acid Cycle,8.885,0.127574
MDH,Mal ↔ OAA,0.457667,0.01365,0.000000,Citric Acid Cycle,6.865,0.204756
CS,OAA + AcCoA → Cit,0.592333,0.008505,0.089804,Citric Acid Cycle,8.885,0.127574
FUM,Suc → Mal,0.523333,0.007572,0.087308,Citric Acid Cycle,7.850,0.113578
ENO,PGA ↔ PEP,1.511000,0.025632,1.532352,Glycolysis/Gluconeogenesis,22.665,0.38448


### Visualise fluxes

In [110]:
# build
# builder = Builder(
#     map_name='iJO1366.Central metabolism',
#     model_name='iJO1366')

builder = Builder(
    map_name='e_coli_core.Core metabolism',
    model_name='e_coli_core')

builder

Builder()

In [111]:
builder.reaction_data = sol_glc.fluxes

### Save the dataframe

In [52]:
# Save as .csv
glc_flux_df.to_csv('../tables/glc_flux_df.csv')

# 1.2 Set acetate flux

In [105]:
# Load the model
# EcN_model = cobra.io.load_json_model('../data/models/CP022686.1_cur_4.7.json')
EcN_model = cobra.io.load_json_model('../data/models/iML1515.json')

# Define the biolog media
medium = EcN_model.medium
medium['EX_glc__D_e'] = 15
    
medium['EX_nh4_e'] = 19 # 19 mM NH4Cl
medium['EX_pi_e'] = 60 # 48mM Na2HPO4 + 22 mM KH2PO4  
medium['EX_so4_e'] = 2 # 2 mM MgSO4
    
medium['EX_mg2_e'] = 2 # 2 mM MgSO4
medium['EX_ca2_e'] = 0.1 # 0.1 mM CaCl2SS
    
medium['EX_na1_e'] = 105 # 48 mM Na2HPO4 + 9 mM NaCl
medium['EX_k_e'] = 22 # 22 mM KH2PO4
medium['EX_cl_e'] = 28 # 9 mM NaCl + 19 mM NH4Cl
    
medium['EX_thm_e'] = 0.38 # 0.1 g/L > 0.1/265.355 x 1000 = 0.376854 mM

# trace elements
value = 0.01
medium['EX_fe3_e'] = value
medium['EX_mn2_e'] = value
medium['EX_fe2_e'] = value
medium['EX_zn2_e'] = value
medium['EX_ni2_e'] = value
medium['EX_cu2_e'] = value
medium['EX_cobalt2_e'] = value
medium['EX_sel_e'] = value
medium['EX_mobd_e'] = value
medium['EX_tungs_e'] = value
medium['EX_slnt_e'] = value
# medium['EX_salchsx_e'] = value

EcN_model.medium = medium

########################################################################## 0.77

# Additionally adapt acetate production
EcN_model.reactions.EX_ac_e.lower_bound = 7.215 # 0.481*15 -- 0.6679
EcN_model.reactions.EX_ac_e.upper_bound = 7.215
# EcN_model.reactions.ACKr.lower_bound = -7.215 # -- 0.6028


# Run parsimonius FBA
sol_glc = cobra.flux_analysis.pfba(EcN_model)

# Export to dataframe
sol_glc_df = sol_glc.to_frame()

In [106]:
sol_glc_df.loc['ACKr']

fluxes          -6.544524
reduced_costs    2.000000
Name: ACKr, dtype: float64

In [107]:
# Merge dataframes
glc_flux_df = pd.merge(flux_df, sol_glc_df, left_index = True, right_index=True)
glc_flux_df['fluxes_corrected'] = glc_flux_df['fluxes']/15 # normalise for 15 glucose
glc_flux_df.drop(columns=['fluxes', 'reduced_costs'], inplace=True) # Do not remove when normalising the 13C data
glc_flux_df

# Alternatively compare the flux with the corrected 13C data
# glc_flux_df['glc_flux_mean_corrected'] = glc_flux_df['glc_flux_mean']*15 # normalise for 15 glucose
# glc_flux_df['glc_flux_sd_corrected'] = glc_flux_df['glc_flux_sd']*15 # normalise for 15 glucose
# glc_flux_df

# Add the subsystem for each reaction and sort
for rxn_id in glc_flux_df.index.values:
    glc_flux_df.loc[rxn_id, 'Subsystem'] = EcN_model.reactions.get_by_id(rxn_id).subsystem
glc_flux_df.sort_values('Subsystem')

,Reaction,glc_flux_mean,glc_flux_sd,fluxes_corrected,Subsystem
ME2,Mal → Pyr + CO2,0.066000,0.008185,0.000000,Anaplerotic Reactions
ME1,Mal → Pyr + CO2,0.066000,0.008185,0.000000,Anaplerotic Reactions
PPC,PEP + CO2 ↔ OAA,0.260000,0.01253,0.227364,Anaplerotic Reactions
AKGDH,AKG → Suc + CO2,0.523333,0.008505,0.243972,Citric Acid Cycle
MDH,Mal ↔ OAA,0.457667,0.01365,0.324065,Citric Acid Cycle
FUM,Suc → Mal,0.523333,0.007572,0.324065,Citric Acid Cycle
ICDHyr,Cit → AKG + CO2,0.592333,0.008505,0.326553,Citric Acid Cycle
CS,OAA + AcCoA → Cit,0.592333,0.008505,0.326553,Citric Acid Cycle
PYK,PEP ↔ Pyr,0.203000,0.020809,0.000000,Glycolysis/Gluconeogenesis
PDH,Pyr → AcCoA + CO2,1.262667,0.01097,1.050731,Glycolysis/Gluconeogenesis


In [108]:
glc_flux_df['diff'] = abs((abs(glc_flux_df['fluxes_corrected']) - abs(glc_flux_df['glc_flux_mean'])) / glc_flux_df['glc_flux_mean'])
print(glc_flux_df['diff'].mean())
glc_flux_df.sort_values('Subsystem')

0.49823436200202004


,Reaction,glc_flux_mean,glc_flux_sd,fluxes_corrected,Subsystem,diff
ME2,Mal → Pyr + CO2,0.066000,0.008185,0.000000,Anaplerotic Reactions,1.000000
ME1,Mal → Pyr + CO2,0.066000,0.008185,0.000000,Anaplerotic Reactions,1.000000
PPC,PEP + CO2 ↔ OAA,0.260000,0.01253,0.227364,Anaplerotic Reactions,0.125525
AKGDH,AKG → Suc + CO2,0.523333,0.008505,0.243972,Citric Acid Cycle,0.533812
MDH,Mal ↔ OAA,0.457667,0.01365,0.324065,Citric Acid Cycle,0.291920
FUM,Suc → Mal,0.523333,0.007572,0.324065,Citric Acid Cycle,0.380768
ICDHyr,Cit → AKG + CO2,0.592333,0.008505,0.326553,Citric Acid Cycle,0.448701
CS,OAA + AcCoA → Cit,0.592333,0.008505,0.326553,Citric Acid Cycle,0.448701
PYK,PEP ↔ Pyr,0.203000,0.020809,0.000000,Glycolysis/Gluconeogenesis,1.000000
PDH,Pyr → AcCoA + CO2,1.262667,0.01097,1.050731,Glycolysis/Gluconeogenesis,0.167848


In [109]:
EcN_model.metabolites.accoa_c.summary()

Percent,Flux,Reaction,Definition
0.00%,0.0005161,PAI2T,coa_c + dhptdd_c --> accoa_c + dhap_c
99.21%,15.76,PDH,coa_c + nad_c + pyr_c --> accoa_c + co2_c + nadh_c
0.79%,0.1253,POR5,coa_c + 2.0 flxso_c + pyr_c <=> accoa_c + co2_c + 2.0 flxr_c + h_c
Percent,Flux,Reaction,Definition
5.16%,-0.82,ACACT1r,2.0 accoa_c <=> aacoa_c + coa_c
2.58%,-0.41,ACACT2r,accoa_c + btcoa_c <=> 3ohcoa_c + coa_c
2.58%,-0.41,ACACT3r,accoa_c + hxcoa_c <=> 3oocoa_c + coa_c
2.58%,-0.41,ACACT4r,accoa_c + occoa_c <=> 3odcoa_c + coa_c
2.58%,-0.41,ACACT5r,accoa_c + dcacoa_c <=> 3oddcoa_c + coa_c
2.02%,-0.3203,ACACT6r,accoa_c + ddcacoa_c <=> 3otdcoa_c + coa_c


What happens to flow towards TCA cycle?
- Extra acetate production via ACALD and ALDD22y - 17.67%
- ACACT various versions (Acetyl-CoA C-acyltransferases) - 18.7% > also present in iML1515
- When acetate flow is high enough, TCA cycle flow is not sufficient


# 2. Gluconate

In [53]:
EcN_model = cobra.io.load_json_model('../data/models/CP022686.1_cur_4.7.json')

# Define the biolog media
medium = EcN_model.medium
medium['EX_glc__D_e'] = 0
medium['EX_glcn_e'] = 15
    
medium['EX_nh4_e'] = 19 # 19 mM NH4Cl
medium['EX_pi_e'] = 60 # 48mM Na2HPO4 + 22 mM KH2PO4  
medium['EX_so4_e'] = 2 # 2 mM MgSO4
    
medium['EX_mg2_e'] = 2 # 2 mM MgSO4
medium['EX_ca2_e'] = 0.1 # 0.1 mM CaCl2SS
    
medium['EX_na1_e'] = 105 # 48 mM Na2HPO4 + 9 mM NaCl
medium['EX_k_e'] = 22 # 22 mM KH2PO4
medium['EX_cl_e'] = 28 # 9 mM NaCl + 19 mM NH4Cl
    
medium['EX_thm_e'] = 0.1 # 0.1 g/L

EcN_model.medium = medium

In [54]:
# Calculate fluxes
sol_glcn = cobra.flux_analysis.pfba(EcN_model) # parsimonious fba
# sol_glcn = EcN_model.optimize() # fba

# Export to dataframe
sol_glcn_df = sol_glcn.to_frame()

In [55]:
# Import the flux file and set index
flux_df = pd.read_excel('../tables/13C_fluxes.xlsx', usecols=(0,1,4,5))
flux_df = flux_df.dropna()
flux_df.set_index('ID', inplace=True)
flux_df.head()

,Reaction,glcn_flux_mean,glcn_flux_sd
ID,,,
AKGDH,AKG → Suc + CO2,0.671417,0.006851
FBA,FBP ↔ GAP + GAP,0.088321,0.00648
TPI,FBP ↔ GAP + GAP,0.088321,0.00648
CS,OAA + AcCoA → Cit,0.711340,0.006851
EDA,6PG → GAP + Pyr,0.808279,0.011769


In [56]:
# Merge the 13C data with the model's flux predictions
glcn_flux_df = pd.merge(flux_df, sol_glcn_df, left_index = True, right_index=True)
glcn_flux_df['fluxes_corrected'] = glcn_flux_df['fluxes']/15 # normalise for 15 gluconate
glcn_flux_df.drop(columns=['fluxes', 'reduced_costs'], inplace=True)

# Add the subsystem for each reaction and sort
for rxn_id in glcn_flux_df.index.values:
    glcn_flux_df.loc[rxn_id, 'Subsystem'] = EcN_model.reactions.get_by_id(rxn_id).subsystem   
glcn_flux_df.sort_values('Subsystem')

,Reaction,glcn_flux_mean,glcn_flux_sd,fluxes_corrected,Subsystem
DMALRED,Mal ↔ OAA,0.665776,0.016548,0.000000,
ME2,Mal → Pyr + CO2,0.005609,0.009716,0.000000,Anaplerotic Reactions
ME1,Mal → Pyr + CO2,0.005609,0.009716,0.000000,Anaplerotic Reactions
PPC,PEP + CO2 ↔ OAA,0.111706,0.009716,0.214430,Anaplerotic Reactions
AKGDH,AKG → Suc + CO2,0.671417,0.006851,0.053246,Citric Acid Cycle
ICDHyr,Cit → AKG + CO2,0.711340,0.006851,0.131130,Citric Acid Cycle
MDH,Mal ↔ OAA,0.665776,0.016548,0.128758,Citric Acid Cycle
CS,OAA + AcCoA → Cit,0.711340,0.006851,0.131130,Citric Acid Cycle
FUM,Suc → Mal,0.671417,0.006851,0.128758,Citric Acid Cycle
ENO,PGA ↔ PEP,0.971385,0.00648,1.368112,Glycolysis/Gluconeogenesis


In [57]:
builder_glcn = Builder(
    map_name='e_coli_core.Core metabolism',
    model_name='e_coli_core')

builder_glcn

Builder()

In [58]:
builder_glcn.reaction_data = sol_glcn.fluxes

### Test: Gluconate with forced ED pathway

In [59]:
# Restrict the flow through the PPP to 1/5th of the substrate
EcN_model.reactions.GND.upper_bound = 3

In [60]:
# Calculate fluxes
sol_glcn_ED = cobra.flux_analysis.pfba(EcN_model)
# sol_glcn = EcN_model.optimize()

# Export to dataframe
sol_glcn_ED_df = sol_glcn_ED.to_frame()

In [61]:
# Merge tables
glcn_flux_ED_df = pd.merge(glcn_flux_df, sol_glcn_ED_df, left_index = True, right_index=True)
glcn_flux_ED_df['fluxes_ED_corrected'] = glcn_flux_ED_df['fluxes']/15 # normalise for 15 gluconate

# Drop unused columns
glcn_flux_ED_df.drop(columns=['fluxes', 'reduced_costs'], inplace=True)

glcn_flux_ED_df

,Reaction,glcn_flux_mean,glcn_flux_sd,fluxes_corrected,Subsystem,fluxes_ED_corrected
AKGDH,AKG → Suc + CO2,0.671417,0.006851,0.053246,Citric Acid Cycle,2.821216e-01
FBA,FBP ↔ GAP + GAP,0.088321,0.00648,0.000000,Glycolysis/Gluconeogenesis,0.000000e+00
TPI,FBP ↔ GAP + GAP,0.088321,0.00648,0.593435,Glycolysis/Gluconeogenesis,6.598588e-02
CS,OAA + AcCoA → Cit,0.711340,0.006851,0.131130,Citric Acid Cycle,3.537476e-01
EDA,6PG → GAP + Pyr,0.808279,0.011769,0.000000,Pentose Phosphate Pathway,8.000000e-01
ENO,PGA ↔ PEP,0.971385,0.00648,1.368112,Glycolysis/Gluconeogenesis,8.587005e-01
FUM,Suc → Mal,0.671417,0.006851,0.128758,Citric Acid Cycle,3.515667e-01
RPI,6PG → R5P + CO2,0.197801,0.019392,-0.386601,Pentose Phosphate Pathway,-1.156546e-01
TKT2,F6P ↔ E4P + E2,0.045930,0.006464,0.291478,Pentose Phosphate Pathway,2.817467e-02
TKT1,R5P + E2 ↔ S7P,0.059300,0.006464,0.319037,Pentose Phosphate Pathway,5.351908e-02


In [62]:
builder_glcn_ED = Builder(
    map_name='e_coli_core.Core metabolism',
    model_name='e_coli_core')

builder_glcn_ED

Builder()

In [63]:
builder_glcn_ED.reaction_data = sol_glcn_ED.fluxes

In [64]:
# Save the dataframe
glcn_flux_ED_df.to_csv('../tables/glcn_flux_ED_df.csv')